# Schema Translation with AgentCore Gateway Interceptor 

### Overview

This notebook shows how to use AgentCore Gateway for **Schema Translation** between existing service contracts and MCP tool schemas, so agents can call your APIs without any custom protocol glue. The goal is to give you a reusable pattern to normalize heterogeneous backend schemas into a clean, agent-friendly interface, while keeping your source systems unchanged.

### Why This Matters

As your agent expands, you may need to:

- Restrict which **tools** certain users can call  
- Control access to **sensitive actions** (cancelOrder, updateOrder, deleteOrder, etc.)  
- Filter or redact **semantic search results** based on user permissions  
- Show users **only the tools they are allowed to see**  
- Enforce custom authorization logic that goes beyond what JWT tokens provide  
- Apply **centralized governance** without modifying individual tools or runtimes  

Gateway interceptors provide a scalable, plug-and-play way to implement these controls **without modifying the agent, the runtime, or the MCP server**.  
You enforce policy **at the Gateway level**, where every request naturally flows through.

### Why Schema Translation Matters

When building AI agents that interact with multiple backend services, schema inconsistencies can create integration complexity and slow down agent development. Schema translation via the AgentCore Gateway allows you to normalize and unify diverse API schemas into a consistent MCP format that your agents can easily consume. 

This approach:

- Eliminates the need to modify existing APIs or backends while exposing them as MCP-compatible tools
- Centralizes protocol translation and schema evolution, reducing maintenance overhead
- Enables seamless integration of REST APIs, Lambda functions, and MCP servers behind a single gateway
- Provides a scalable, maintainable way to keep your agent’s view of tools clean and consistent

Gateway interceptors provide a scalable, plug-and-play way to implement schema translation **without modifying the agent, the runtime, or the MCP server**.  
You enforce policy **at the Gateway level**, where every request naturally flows through.

---

### What This Tutorial Covers

You will implement Schema Translation by:

1. 🛠️ **Create AgentCore Gateway with Request Gateway Interceptor** 
The Request interceptor has logic to transform the schemas in the request and to add any additional parameters. 

2. 🔍 **Register tools with the AgentCore Gateway**  
Register the inventory tool with the AgentCore Gateway 

3. 📋 **Invoke tool**  
When the tool is invoked,the request is intercepted and the schema translation is performed. 

---

### Why Use Gateway Interceptors?

Gateway interceptors allow you to add logic for:

- **Performing Schema Translation**
Seamlessly convert request and response schemas between heterogeneous backend APIs and the MCP format.

- **Normalizing Tool Interfaces**
Create a consistent, agent-friendly schema abstraction across diverse services without changing underlying APIs.

- **Adapt Protocols and Data Formats**
Modify requests or responses to bridge differences in expected parameters, field names, or data structures.

- **Centralize Translation Logic**
Manage schema evolution and transformations in one place to simplify maintenance and improve scalability.

Because interceptors operate at the Gateway layer, they enable consistent schema handling for any connected MCP server or runtime without altering individual tool implementations.

![schema-translation](images/schema-translation-request-interceptor.png)

---

### Tutorial Details

| Information              | Details                                                                                         |
|--------------------------|-------------------------------------------------------------------------------------------------|
| **Tutorial type**        | Interactive                                                                                     |
| **AgentCore components** | AgentCore Gateway, Gateway Interceptors (Request)                                               |
| **Gateway Target type**  | Lambda Function                                                                                 |
| **Interceptor types**    | AWS Lambda (request)                                                                            |
| **Inbound Auth IdP**     | Amazon Cognito (CUSTOM\_JWT authorizer)                                                         |
| **Access Control**       | IAM to invoke lambda tool                                                                       |
| **Tutorial components**  | Gateway, Cognito, Gateway Interceptors, MCP tools (lambda)                                      |
| **Tutorial vertical**    | Cross-vertical                                                                                  |
| **Example complexity**   | Easy–Intermediate                                                                               |
| **SDK used**             | boto3                                                                                           |

---

### Prerequisites

To execute this tutorial you will need:

- Jupyter notebook (Python kernel)
- AWS credentials with permissions for:
  - Lambda
  - IAM
  - Cognito
  - AgentCore services (control plane + runtime)
- Python 3.13 or higher
- Basic understanding of AWS Lambda, IAM roles, Cognito, and AgentCore Gateway

> ⚠️ **Note:** The Cleanup section at the end deletes the AWS resources created by this tutorial (Gateway, Lambdas, IAM roles, etc.). Only run it when you’re ready to tear everything down.

---
## Installing Libraries

In [ ]:
!pip install -r requirements.txt

## Import Required Libraries

In [ ]:
import boto3
import json
import time
import zipfile
import io
import sys
from pathlib import Path
from datetime import datetime
from botocore.exceptions import ClientError

# Add parent directory to path for utils
utils_dir = Path.cwd().parent
sys.path.insert(0, str(utils_dir))

import utils

print("✓ Libraries imported")

# Generate unique identifier for this deployment
DEPLOYMENT_ID = datetime.now().strftime('%Y%m%d-%H%M%S')
print(f"\nDeployment ID: {DEPLOYMENT_ID}")

## Setting up Variables

In [ ]:
# Configuration

LAMBDA_REGION = "us-east-1"  
REQUEST_LAMBDA_FUNCTION_NAME = f"request_interceptor-lambda-{DEPLOYMENT_ID}"
LAMBDA_ROLE_NAME = f"interceptor-lambda-role-{DEPLOYMENT_ID}"
GATEWAY_NAME = f"interceptor-gateway-{DEPLOYMENT_ID}"

print("Configuration:")
print(f" Request Lambda Function: {REQUEST_LAMBDA_FUNCTION_NAME}")
print(f"  Lambda Role: {LAMBDA_ROLE_NAME}")
print(f"  Gateway Name: {GATEWAY_NAME}")
print(f"  Region: {LAMBDA_REGION}")


## Create IAM Role for Lambda

In [ ]:
# Create IAM role for Lambda
print("Creating IAM role...")

iam_client = boto3.client('iam')

# Trust policy for Lambda
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {"Service": "lambda.amazonaws.com"},
            "Action": "sts:AssumeRole"
        }
    ]
}

try:
    role_response = iam_client.create_role(
        RoleName=LAMBDA_ROLE_NAME,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description='Role for AgentCore Lambda Interceptor'
    )
    
    LAMBDA_ROLE_ARN = role_response['Role']['Arn']
    print(f"✓ IAM Role created: {LAMBDA_ROLE_NAME}")
    print(f"  ARN: {LAMBDA_ROLE_ARN}")
    
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print(f"⚠ Role already exists: {LAMBDA_ROLE_NAME}")
        role_response = iam_client.get_role(RoleName=LAMBDA_ROLE_NAME)
        LAMBDA_ROLE_ARN = role_response['Role']['Arn']
        print(f"  ARN: {LAMBDA_ROLE_ARN}")
    else:
        raise

# Attach basic Lambda execution policy
print("  Attaching Lambda execution policy...")
iam_client.attach_role_policy(
    RoleName=LAMBDA_ROLE_NAME,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

# Wait for role to propagate
print("  Waiting for IAM role to propagate...")
time.sleep(10)
print("  ✓ IAM role ready")

## Creating Request Lambda Deployment Zip File

In [ ]:
# Verify Lambda code uses correct format
print("Verifying Lambda Request interceptor code...")

lambda_code_path = Path('src/lambda/st_request_interceptor.py')

if not lambda_code_path.exists():
    print(f"✗ Lambda code not found: {lambda_code_path}")
    raise FileNotFoundError(f"Missing {lambda_code_path}")

with open(lambda_code_path, 'r') as f:
    lambda_code = f.read()

# Create deployment package
print("Creating deployment package...")
zip_buffer = io.BytesIO()
with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
    zip_file.writestr('lambda_function.py', lambda_code)

zip_buffer.seek(0)
deployment_package = zip_buffer.read()
print(f"✓ Package size: {len(deployment_package)} bytes")

## Creating Lambda Request Interpretor

In [ ]:
# Create Lambda function
print("Creating Lambda Request interceptor function...")

lambda_client = boto3.client('lambda', region_name=LAMBDA_REGION)

try:
    response = lambda_client.create_function(
        FunctionName=REQUEST_LAMBDA_FUNCTION_NAME,
        Runtime='python3.13',
        Role=LAMBDA_ROLE_ARN,
        Handler='lambda_function.lambda_handler',
        Code={'ZipFile': deployment_package},
        Description='AgentCore Response Lambda Interceptor to mask sensitive data ',
        Timeout=30,
        MemorySize=256
    )
    
    REQUEST_LAMBDA_ARN = response['FunctionArn']
    print(f"✓ Lambda created: {REQUEST_LAMBDA_FUNCTION_NAME}")
    print(f"  ARN: {REQUEST_LAMBDA_ARN}")
    print(f"  Runtime: {response['Runtime']}")
    print(f"  Memory: {response['MemorySize']} MB")
    
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceConflictException':
        print(f"⚠ Lambda already exists: {REQUEST_LAMBDA_FUNCTION_NAME}")
        response = lambda_client.get_function(FunctionName=REQUEST_LAMBDA_FUNCTION_NAME)
        REQUEST_LAMBDA_ARN = response['Configuration']['FunctionArn']
        print(f"  ARN: {REQUEST_LAMBDA_ARN}")
    else:
        raise

## Granting Gateway permission to invoke the interceptor Lambda

In [ ]:
# Grant Gateway permission to invoke the interceptor Lambda
print("\n🔐 Granting Gateway permission to invoke Lambda...")

# Get AWS account ID
sts_client = boto3.client('sts')
ACCOUNT_ID = sts_client.get_caller_identity()['Account']

try:
    lambda_client.add_permission(
        FunctionName=REQUEST_LAMBDA_FUNCTION_NAME,
        StatementId='AllowGatewayInvoke',
        Action='lambda:InvokeFunction',
        Principal='bedrock-agentcore.amazonaws.com',
        SourceArn=f'arn:aws:bedrock-agentcore:{LAMBDA_REGION}:{ACCOUNT_ID}:gateway/*'
    )
    print(f"✓ Gateway invoke permission added to Lambda")
    print(f"  Principal: bedrock-agentcore.amazonaws.com")
    print(f"  Source: arn:aws:bedrock-agentcore:{LAMBDA_REGION}:{ACCOUNT_ID}:gateway/*")
    
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceConflictException':
        print(f"⚠ Permission already exists (this is fine)")
    else:
        print(f"⚠ Error adding permission: {e}")
        raise


## Creating Cognito User Pool for Inbound Authentication

In [ ]:
# Create Cognito User Pool and Client for Gateway authentication
print("Creating Cognito User Pool and Client...")

cognito_client = boto3.client('cognito-idp', region_name='us-east-1')

# Create User Pool
USER_POOL_NAME = f"gateway-pool-{DEPLOYMENT_ID}"

try:
    pool_response = cognito_client.create_user_pool(
        PoolName=USER_POOL_NAME,
        Policies={'PasswordPolicy': {
            'MinimumLength': 8,
            'RequireUppercase': False,
            'RequireLowercase': False,
            'RequireNumbers': False,
            'RequireSymbols': False
        }}
    )
    
    USER_POOL_ID = pool_response['UserPool']['Id']
    print(f"✓ User Pool created: {USER_POOL_NAME}")
    print(f"  Pool ID: {USER_POOL_ID}")
    
except ClientError as e:
    print(f"⚠ Error creating user pool: {e}")
    raise

# Create User Pool Domain (required for OAuth)
POOL_DOMAIN = f"interceptor-{DEPLOYMENT_ID.replace('_', '-').lower()}"

try:
    cognito_client.create_user_pool_domain(Domain=POOL_DOMAIN, UserPoolId=USER_POOL_ID)
    print(f"✓ User Pool Domain created: {POOL_DOMAIN}")
except ClientError as e:
    if 'Domain already exists' in str(e):
        print(f"⚠ Domain already exists: {POOL_DOMAIN}")
    else:
        print(f"⚠ Error creating domain: {e}")

# Create Resource Server with custom scope
try:
    cognito_client.create_resource_server(
        UserPoolId=USER_POOL_ID,
        Identifier='gateway',
        Name='Gateway Resource Server',
        Scopes=[{'ScopeName': 'tools', 'ScopeDescription': 'Access to gateway tools'}]
    )
    print(f"✓ Resource Server created with scope: gateway/tools")
except ClientError as e:
    print(f"⚠ Resource server error: {e}")

# Wait for resource server
print("  Waiting for resource server to propagate...")
time.sleep(3)

# Create User Pool Client with client credentials flow
CLIENT_NAME = f"gateway-client-{DEPLOYMENT_ID}"

try:
    client_response = cognito_client.create_user_pool_client(
        UserPoolId=USER_POOL_ID,
        ClientName=CLIENT_NAME,
        GenerateSecret=True,
        ExplicitAuthFlows=[],
        AllowedOAuthFlows=['client_credentials'],
        AllowedOAuthScopes=['gateway/tools'],
        AllowedOAuthFlowsUserPoolClient=True,
        SupportedIdentityProviders=[]
    )
    
    CLIENT_ID = client_response['UserPoolClient']['ClientId']
    CLIENT_SECRET = client_response['UserPoolClient']['ClientSecret']
    
    print(f"✓ User Pool Client created: {CLIENT_NAME}")
    print(f"  Client ID: {CLIENT_ID}")
    print(f"  Client Secret: {CLIENT_SECRET[:20]}...")
    
    # Construct OAuth URLs
    COGNITO_DOMAIN = f"https://{POOL_DOMAIN}.auth.us-east-1.amazoncognito.com"
    DISCOVERY_URL = f"https://cognito-idp.us-east-1.amazonaws.com/{USER_POOL_ID}/.well-known/openid-configuration"
    TOKEN_URL = f"{COGNITO_DOMAIN}/oauth2/token"
    
    print(f"\n✓ OAuth Configuration:")
    print(f"  Discovery URL: {DISCOVERY_URL}")
    print(f"  Token URL: {TOKEN_URL}")
    print(f"  Scope: gateway/tools")
    
except ClientError as e:
    print(f"✗ Error creating client: {e}")
    raise

## Creating Gateway Client using Boto3

In [ ]:
# Initialize Boto3 client for bedrock-agentcore-control
print("Initializing Boto3 Gateway client...")

gateway_client = boto3.client('bedrock-agentcore-control', region_name=LAMBDA_REGION)

print(f"✓ Gateway client initialized for region: {LAMBDA_REGION}")

## Creating AgentCore Gateway with Lambda Request Interpretor

In [ ]:
# Create Gateway with Lambda interceptors using signed HTTP requests
print("Creating Gateway with Lambda REQUEST and RESPONSE interceptors...")

# First, create an IAM role for the Gateway
iam_client = boto3.client('iam')

gateway_trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock-agentcore.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

GATEWAY_ROLE_NAME = f"gateway-role-{DEPLOYMENT_ID}"

try:
    gateway_role_response = iam_client.create_role(
        RoleName=GATEWAY_ROLE_NAME,
        AssumeRolePolicyDocument=json.dumps(gateway_trust_policy),
        Description='IAM role for Bedrock AgentCore Gateway'
    )
    
    GATEWAY_ROLE_ARN = gateway_role_response['Role']['Arn']
    print(f"✓ Gateway IAM role created: {GATEWAY_ROLE_NAME}")
    print(f"  ARN: {GATEWAY_ROLE_ARN}")
    
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print(f"⚠ Role already exists: {GATEWAY_ROLE_NAME}")
        gateway_role_response = iam_client.get_role(RoleName=GATEWAY_ROLE_NAME)
        GATEWAY_ROLE_ARN = gateway_role_response['Role']['Arn']
        print(f"  ARN: {GATEWAY_ROLE_ARN}")
    else:
        raise

# Attach necessary policies to the Gateway role
print("  Attaching policies to Gateway role...")
try:
    # Attach Lambda invoke policy
    lambda_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "lambda:InvokeFunction",
                "Resource": "*"
            }
        ]
    }
    
    iam_client.put_role_policy(
        RoleName=GATEWAY_ROLE_NAME,
        PolicyName='LambdaInvokePolicy',
        PolicyDocument=json.dumps(lambda_policy)
    )
    print("  ✓ Lambda invoke policy attached")
except Exception as e:
    print(f"  ⚠ Policy attach error: {e}")

# Wait for Gateway role to propagate
print("  Waiting for Gateway role to propagate...")
time.sleep(10)

# # Define ARNs for both interceptors
# RESPONSE_LAMBDA_ARN = RESPONSE_LAMBDA_ARN  # Using the existing LAMBDA_ARN for response interceptor
# REQUEST_LAMBDA_ARN = REQUEST_LAMBDA_ARN  # You need to define this variable earlier in your code

# Create Gateway using signed HTTP request with interceptor configuration
print(f"\n  Creating Gateway with signed HTTP request and both REQUEST interceptors:")
print(f"    Name: {GATEWAY_NAME}")
print(f"    Protocol: MCP")
print(f"    Auth: CUSTOM_JWT (Cognito)")
print(f"    Request Interceptor: {REQUEST_LAMBDA_ARN}")


gateway_control_plane_url = f"https://bedrock-agentcore-control.{LAMBDA_REGION}.amazonaws.com/gateways"

try:
    print(f"\n Creating gateway using {gateway_control_plane_url}")

    gateway_response = gateway_client.create_gateway(
        name=GATEWAY_NAME,
        protocolType="MCP",
        protocolConfiguration={
            "mcp": {
                "supportedVersions": ["2025-03-26"]
            }
        },

        interceptorConfigurations=[
            {
                "interceptor": {
                    "lambda": {
                        "arn": REQUEST_LAMBDA_ARN
                    }
                },
                "interceptionPoints": ["REQUEST"],  # Intercept requests for schema translation
                "inputConfiguration": {
                    "passRequestHeaders": True  # Pass Agent-ID header to interceptor
                }
            }
        ],

        authorizerType="CUSTOM_JWT",
        authorizerConfiguration={
            "customJWTAuthorizer": {
                "discoveryUrl": DISCOVERY_URL,
                "allowedClients": [CLIENT_ID]
            }
        },

        roleArn=GATEWAY_ROLE_ARN
    )
    
    status_code = gateway_response.get("ResponseMetadata", {}).get("HTTPStatusCode")
    if status_code not in [200, 202]:
        print(f"\n✗ Failed to create Gateway: {status_code}")
        print(f"  Response: {gateway_response.text}")
        raise Exception(f"Gateway creation failed: {gateway_response.text}")
    
    # gateway_data = gateway_response.json()
    GATEWAY_ID = gateway_response.get('gatewayId')
    
    print(f"\n✓ Gateway created successfully with REQUEST and RESPONSE interceptors")
    print(f"  ID: {GATEWAY_ID}")
    print(f"  Status: {gateway_response.get('status', 'CREATING')}")
    print(f"  Request Interceptor Lambda: {REQUEST_LAMBDA_ARN}")
    
    # Verify interceptor configuration in response
    if 'interceptorConfigurations' in gateway_response and gateway_response['interceptorConfigurations']:
        print(f"  ✓ Interceptor configuration confirmed in response!")
        print(f"    Interceptors: {len(gateway_response['interceptorConfigurations'])}")
        
        # Log details of each interceptor
        for i, config in enumerate(gateway_response['interceptorConfigurations']):
            points = config.get('interceptionPoints', [])
            print(f"    Interceptor {i+1}: {', '.join(points)}")
    else:
        print(f"  ⚠ Warning: No interceptor configuration in response")
    
except Exception as e:
    print(f"\n✗ Failed to create Gateway: {e}")
    raise


## Waiting for the Gateway to be ready

In [ ]:
# Wait for Gateway to be ready using signed requests
print("\nWaiting for Gateway to be ready...")

# get_gateway_url = f"{gateway_control_plane_url}/{GATEWAY_ID}"

max_attempts = 30
for attempt in range(max_attempts):
    try:
        response = gateway_client.get_gateway(gatewayIdentifier=GATEWAY_ID)
        status_code = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

        if status_code == 200:
            # gateway_info = response.json()
            status = response.get('status', 'UNKNOWN')
            
            print(f"  [{attempt + 1}/{max_attempts}] Status: {status}")
            
            if status == 'READY':
                GATEWAY_URL = response.get('gatewayUrl')
                print(f"\n✓ Gateway is ready!")
                print(f"  URL: {GATEWAY_URL}")
                
                # Show interceptor configuration
                if 'interceptorConfigurations' in response:
                    interceptor_configs = response['interceptorConfigurations']
                    print(f"\n  Interceptor Configuration:")
                    for idx, config in enumerate(interceptor_configs):
                        print(f"    [{idx}] Interception Points: {config.get('interceptionPoints', [])}")
                        print(f"    [{idx}] Lambda ARN: {config.get('interceptor', {}).get('lambda', {}).get('arn', 'N/A')}")
                        print(f"    [{idx}] Pass Headers: {config.get('inputConfiguration', {}).get('passRequestHeaders', False)}")
                break
            elif status == 'FAILED':
                print(f"\n✗ Gateway creation failed")
                print(f"  Details: {response}")
                raise Exception("Gateway failed")
        else:
            print(f"  [{attempt + 1}/{max_attempts}] HTTP Error: {response.status_code}")
    except Exception as e:
        print(f"  [{attempt + 1}/{max_attempts}] Error: {e}")
    
    time.sleep(10)
else:
    print(f"\n⚠ Timeout waiting for Gateway")
    raise Exception("Gateway timeout")


## Verifying Interceptor Configuration on Gateway

In [ ]:
# ⭐ CRITICAL: Verify interceptor configuration is actually stored
print("\n🔍 Verifying interceptor configuration on Gateway...")
print("-" * 60)

if 'interceptorConfigurations' not in response:
    print("\n❌ ERROR: Gateway does NOT have interceptor configured!")
    print("The interceptorConfigurations parameter was not accepted.")
    print("\nGateway Info:")
    print(json.dumps(response, indent=2, default=str))
    raise Exception("Interceptor not configured on Gateway - this explains why filtering isn't working!")

# If we get here, interceptor IS configured
interceptor_configs = response['interceptorConfigurations']
print(f"✓ Interceptor configuration found!")
print(f"  Number of interceptors: {len(interceptor_configs)}")

for idx, config in enumerate(interceptor_configs):
    print(f"\n  Interceptor [{idx}]:")
    print(f"    Interception Points: {config.get('interceptionPoints', [])}")
    print(f"    Lambda ARN: {config.get('interceptor', {}).get('lambda', {}).get('arn', 'N/A')}")
    print(f"    Pass Headers: {config.get('inputConfiguration', {}).get('passRequestHeaders', False)}")
    
    # Verify it matches what we configured
    configured_arn = config.get('interceptor', {}).get('lambda', {}).get('arn', '')
    if configured_arn == REQUEST_LAMBDA_ARN:
        print(f"    ✓ Lambda ARN matches our interceptor")
    else:
        print(f"    ⚠ Lambda ARN mismatch!")
        print(f"      Expected: {REQUEST_LAMBDA_ARN}")
        print(f"      Got: {configured_arn}")
    
    # Verify passRequestHeaders is enabled
    pass_headers = config.get('inputConfiguration', {}).get('passRequestHeaders', False)
    if pass_headers:
        print(f"    ✓ Request headers will be passed to interceptor")
    else:
        print(f"    ⚠ WARNING: passRequestHeaders is FALSE - Agent-ID won't be passed!")

print("\n" + "-" * 60)
print("✅ Interceptor configuration verification complete")


## Register Sample Inventory with Gateway

In [ ]:
# Deploy real tool Lambdas and register as Gateway targets using SIGNED HTTP requests
print("="*80)
print("Deploying Real Tool Lambdas and Registering with Gateway")
print("="*80)

import sys
import importlib

# Step 1: Deploy tool Lambda functions
print("\n📦 Step 1: Deploying tool Lambda functions...")
print("-" * 60)

# Import tool modules with RELOAD to get latest changes
sys.path.insert(0, str(Path.cwd()))

# Force reload modules to get updated TOOL_DEFINITION (without enum)
from src.tools import inventory_tool

# Reload all modules to ensure we get the latest TOOL_DEFINITION
inventory_tool = importlib.reload(inventory_tool)

print("✓ Tool modules reloaded with updated schemas")

# Create IAM role for tool Lambdas
TOOL_ROLE_NAME = f"tool-lambda-role-{DEPLOYMENT_ID}"

try:
    tool_role_response = iam_client.create_role(
        RoleName=TOOL_ROLE_NAME,
        AssumeRolePolicyDocument=json.dumps({
            "Version": "2012-10-17",
            "Statement": [{"Effect": "Allow", "Principal": {"Service": "lambda.amazonaws.com"}, "Action": "sts:AssumeRole"}]
        }),
        Description='Role for tool Lambda functions'
    )
    TOOL_ROLE_ARN = tool_role_response['Role']['Arn']
    print(f"✓ Tool IAM role created: {TOOL_ROLE_NAME}")
    
    # Attach basic execution policy
    iam_client.attach_role_policy(
        RoleName=TOOL_ROLE_NAME,
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
    )
    time.sleep(15)  # Wait for role to propagate
    
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        tool_role_response = iam_client.get_role(RoleName=TOOL_ROLE_NAME)
        TOOL_ROLE_ARN = tool_role_response['Role']['Arn']
        print(f"⚠ Tool role already exists: {TOOL_ROLE_NAME}")
    else:
        raise

# Additional wait to ensure IAM is fully propagated
print("⏳ Waiting 15 seconds for IAM role propagation...")
time.sleep(15)
print("✓ IAM role should be ready")

# Deploy each tool Lambda
tools_to_deploy = [
    ('inventory_tool', inventory_tool)
]

deployed_tools = []

for tool_name, tool_module in tools_to_deploy:
    print(f"\n  Deploying {tool_name}...")
    
    # Create ZIP
    tool_code_path = Path(tool_module.__file__)
    with open(tool_code_path, 'r') as f:
        tool_code = f.read()
    
    zip_buf = io.BytesIO()
    with zipfile.ZipFile(zip_buf, 'w', zipfile.ZIP_DEFLATED) as zf:
        zf.writestr('lambda_function.py', tool_code)
    zip_buf.seek(0)
    
    function_name = f"{tool_name.replace('_', '-')}-{DEPLOYMENT_ID}"
    
    try:
        response = lambda_client.create_function(
            FunctionName=function_name,
            Runtime='python3.13',
            Role=TOOL_ROLE_ARN,
            Handler='lambda_function.lambda_handler',
            Code={'ZipFile': zip_buf.read()},
            Timeout=30,
            MemorySize=256,
            Environment={'Variables': {'TOOL_NAME': tool_name}}
        )
        lambda_arn = response['FunctionArn']
        print(f"    ✓ Created: {function_name}")
        
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceConflictException':
            response = lambda_client.get_function(FunctionName=function_name)
            lambda_arn = response['Configuration']['FunctionArn']
            print(f"    ⚠ Already exists: {function_name}")
        else:
            raise
    
    # Get tool definition (now reloaded without enum)
    tool_definition = getattr(tool_module, 'TOOL_DEFINITION', {
        "name": tool_name,
        "description": f"{tool_name} function"
    })
    
    # Verify no enum in tool_definition
    tool_def_str = json.dumps(tool_definition)
    if '"enum"' in tool_def_str:
        print(f"    ⚠ WARNING: Tool definition still contains 'enum' - module may not have reloaded!")
        print(f"    Tool definition: {tool_def_str[:200]}...")
    
    deployed_tools.append({
        'tool_name': tool_name,
        'function_name': function_name,
        'lambda_arn': lambda_arn,
        'tool_definition': tool_definition
    })

print(f"\n✓ Deployed {len(deployed_tools)} tool Lambdas")

# Step 2: Register tools as Gateway targets using SIGNED HTTP REQUESTS (for gamma endpoint)
print(f"\n🎯 Step 2: Registering tools as Gateway targets with signed HTTP requests...")
print(f"  Gateway endpoint: {gateway_control_plane_url}")
print("-" * 60)

created_targets = []

for tool in deployed_tools:
    print(f"\n  Registering {tool['tool_name']}...")
 
    try:
        # Create target using signed HTTP request

        response = gateway_client.create_gateway_target(
            gatewayIdentifier=GATEWAY_ID,
            name=f"{tool['tool_name'].replace('_', '-')}-target",
            targetConfiguration={
                "mcp": {
                    "lambda": {
                        "lambdaArn": tool["lambda_arn"],
                        "toolSchema": {
                            "inlinePayload": [
                                tool["tool_definition"]
                            ]
                        }
                    }
                }
            },
            credentialProviderConfigurations=[
                {
                    "credentialProviderType": "GATEWAY_IAM_ROLE"
                }
            ]
        )

        status_code = response.get("ResponseMetadata", {}).get("HTTPStatusCode")
        if status_code not in [200, 202]:
            print(f"    ✗ Failed to create target: {response.status_code}")
            print(f"    Response: {response.text}")
            continue
        
        # target_data = response.json()
        target_id = response['targetId']
        print(f"    ✓ Target created: {target_id}")
        
        # Wait for target to be READY using signed requests
        print(f"    Waiting for target to be READY...")
        
        for attempt in range(18):  # 3 minutes max
            try:
                response = gateway_client.get_gateway_target(gatewayIdentifier=GATEWAY_ID, targetId=target_id)
                status_code = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

                
                if status_code == 200:
                    # status_data = status_response.json()
                    status = response.get('status', 'UNKNOWN')
                    
                    if status == 'READY':
                        print(f"    ✓ Target is READY")
                        created_targets.append({
                            'tool_name': tool['tool_name'],
                            'target_id': target_id,
                            'lambda_arn': tool['lambda_arn'],
                            'status': 'READY'
                        })
                        break
                    elif status == 'FAILED':
                        print(f"    ✗ Target FAILED")
                        print(f"    Failure details: {json.dumps(status_data, indent=6, default=str)}")
                        break
                else:
                    print(f"    Status check error: HTTP {status_response.status_code}")
                
            except Exception as e:
                print(f"    Status check error: {e}")
            
            time.sleep(10)
            
    except Exception as e:
        print(f"    ✗ Failed to create target: {e}")

# Summary
print(f"\n{'='*80}")
print(f"Summary:")
print(f"  • Tool Lambdas deployed: {len(deployed_tools)}")
print(f"  • Gateway targets created: {len(created_targets)}")
print(f"{'='*80}\n")

for target in created_targets:
    print(f"  ✓ {target['tool_name']}: {target['target_id']} ({target['status']})")

if len(created_targets) < len(deployed_tools):
    print(f"\n⚠ Warning: Not all targets were created successfully")
    print(f"  Some tools may not be available through the Gateway")
else:
    print(f"\n✅ All tools are registered and ready!")

# Store for cleanup
DEPLOYED_TOOL_FUNCTIONS = [t['function_name'] for t in deployed_tools]
CREATED_TARGET_IDS = [t['target_id'] for t in created_targets]


## Listing all Tools on the AgentCore Gateway

In [ ]:
import requests
import json


def fetch_access_token(client_id, client_secret, token_url):
  response = requests.post(
    token_url,
    data="grant_type=client_credentials&client_id={client_id}&client_secret={client_secret}".format(client_id=client_id, client_secret=client_secret),
    headers={'Content-Type': 'application/x-www-form-urlencoded'}
  )

  return response.json()['access_token']

def list_tools(gateway_url, access_token):
  headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {access_token}"
  }

  payload = {
      "jsonrpc": "2.0",
      "id": "list-tools-request",
      "method": "tools/list"
  }

  response = requests.post(gateway_url, headers=headers, json=payload)
  return response.json()

access_token = fetch_access_token(CLIENT_ID, CLIENT_SECRET, TOKEN_URL)
tools = list_tools(GATEWAY_URL, access_token)
print(json.dumps(tools, indent=2))

## Invoking Inventory Tool with Arguments on AgentCore Gateway

In [ ]:
# Helper function to invoke a specific tool through Gateway with REAL OAuth token
def invoke_tool_through_gateway(gateway_url, tool_name, arguments):
    """Make real MCP request to Gateway to invoke a specific tool with real OAuth token from Cognito."""
    import requests as req

    gateway_target_name = "inventory-tool-target"  # Set gateway target name here
    tool_name = "inventory_tool"

    # print(f"\n🌐 Invoking tool '{tool_name}' on agent: {agent_id}")
    print(f"   Gateway URL: {gateway_url}")

    # Get REAL OAuth token from Cognito using the credentials created earlier
    print(f"   Getting real OAuth token from Cognito...")
    print(f"   Using User Pool: {USER_POOL_ID}")
    print(f"   Using Client: {CLIENT_ID}")
    print(f"   Using Client: {CLIENT_SECRET}")

    try:
        # Get token using client credentials flow
        print(f"Requesting token from: {TOKEN_URL}")

        token_response = req.post(
            TOKEN_URL,
            headers={'Content-Type': 'application/x-www-form-urlencoded'},
            data={
                'grant_type': 'client_credentials',
                'client_id': CLIENT_ID,
                'client_secret': CLIENT_SECRET,
                'scope': 'gateway/tools'
            }
        )

        if token_response.status_code == 200:
            token_data = token_response.json()
            oauth_token = token_data['access_token']
            print(oauth_token)
            print(f"   ✓ Got real OAuth token from Cognito")
        else:
            error_msg = f"Failed to get token: {token_response.status_code} - {token_response.text}"
            print(f"   ✗ {error_msg}")
            return None, error_msg

    except Exception as e:
        error_msg = f"Token retrieval error: {e}"
        print(f"   ✗ {error_msg}")
        return None, error_msg

    # Default to empty arguments if none provided
    if arguments is None:
        arguments = {}

    # Construct MCP request for tools/call with tool name and inputs
    mcp_request = {
        "jsonrpc": "2.0",
        "method": "tools/call",
        "id": 1,
        "params": {
            "name": f"{gateway_target_name}___{tool_name}",
            "arguments": arguments
        }
    }

    headers = {
        "Authorization": f"Bearer {oauth_token}",
        "Content-Type": "application/json"
    }

    try:
        gateway_url = GATEWAY_URL
        response = req.post(gateway_url, headers=headers, json=mcp_request, timeout=30)

        if response.status_code == 200:
            data = response.json()
            print(f"   ✓ Tool '{tool_name}' invoked successfully")
            return data, None
        else:
            error = f"Status {response.status_code}: {response.text}"
            print(f"   ✗ {error}")
            return None, error
    except Exception as e:
        error = str(e)
        print(f"   ✗ Error: {error}")
        return None, error


## Invoke tool with an argument

- When tool is invoked using "item_id" argument, the lambda request interceptor changes the schema from "item_id" to "itemId"
- Also the request interceptor adds in an additional argument "location" as "East Warehouse" by default if location is not provided

In [ ]:
# Call the function to invoke the tool through the gateway
arguments = {"item_id": "ITEM-12345"}
result, error = invoke_tool_through_gateway(GATEWAY_URL, tool_name, arguments)

if error:
    print(f"Error invoking tool: {error}")
else:
    print("Tool invocation response:")
    print(result)

## Invoke tool with multiple arguments

- When tool is invoked using "item_id" argument, the lambda request interceptor changes the schema from "item_id" to "itemId"
- since location value is provided by the user, this time it will be passed to the tool and the default value wont be used

In [ ]:
# Call the function to invoke the tool through the gateway
arguments = {
    "item_id": "ITEM-12345",
    "location": "West Warehouse"
    }
result, error = invoke_tool_through_gateway(GATEWAY_URL, tool_name, arguments)

if error:
    print(f"Error invoking tool: {error}")
else:
    print("Tool invocation response:")
    print(result)

---

## Cleanup - Delete All Resources

⚠️ **WARNING: This will DELETE all resources created above!**

Only run this section if you want to clean up everything.

In [ ]:
# Cleanup - Delete all created resources using utils
print("Starting cleanup...")

# 1. Delete gateway targets
if 'CREATED_TARGET_IDS' in globals() and 'GATEWAY_ID' in globals():
    utils.delete_gateway_targets(gateway_client, GATEWAY_ID, CREATED_TARGET_IDS)
    # Wait for target deletions to complete before deleting gateway
    time.sleep(5)

# 2. Delete gateway
if 'GATEWAY_ID' in globals():
    utils.delete_gateway(gateway_client, GATEWAY_ID)
    print("✓ Deleted gateway")

# 3. Delete Lambda functions (tools + interceptor)
lambda_functions_to_delete = []
if 'DEPLOYED_TOOL_FUNCTIONS' in globals():
    lambda_functions_to_delete.extend(DEPLOYED_TOOL_FUNCTIONS)
if 'REQUEST_LAMBDA_FUNCTION_NAME' in globals():
    lambda_functions_to_delete.append(REQUEST_LAMBDA_FUNCTION_NAME)

if lambda_functions_to_delete:
    utils.delete_lambda_functions(lambda_functions_to_delete, LAMBDA_REGION)

# 4. Delete IAM roles
if 'LAMBDA_ROLE_NAME' in globals():
    utils.delete_iam_role(LAMBDA_ROLE_NAME)
if 'DEPLOYMENT_ID' in globals():
    utils.delete_iam_role(f"tool-lambda-role-{DEPLOYMENT_ID}")
    utils.delete_iam_role(f"agentcore-{GATEWAY_NAME}-role")

# 5. Delete Cognito user pool
if 'USER_POOL_ID' in globals():
    utils.delete_cognito_user_pool(USER_POOL_ID, LAMBDA_REGION)

print("\n✓ Cleanup complete!")



## Delete Gateway

In [ ]:
# Delete Gateway and Targets using Boto3
print("\nDeleting Gateway and targets...")

try:
    # First delete all targets
    if 'CREATED_TARGET_IDS' in globals() and CREATED_TARGET_IDS:
        print(f"  Deleting {len(CREATED_TARGET_IDS)} targets...")
        
        for target_id in CREATED_TARGET_IDS:
            try:
                gateway_client.delete_gateway_target(
                    gatewayIdentifier=GATEWAY_ID,
                    targetId=target_id
                )
                print(f"    ✓ Target deleted: {target_id}")
            except ClientError as e:
                print(f"    ⚠ Error deleting target {target_id}: {e}")
        
        time.sleep(5)  # Wait for targets to be deleted
    
    # Then delete Gateway
    try:
        gateway_client.delete_gateway(gatewayIdentifier=GATEWAY_ID)
        print(f"  ✓ Gateway deleted: {GATEWAY_ID}")
    except ClientError as e:
        print(f"  ⚠ Error deleting gateway: {e}")
        
except Exception as e:
    print(f"  ⚠ Error during Gateway cleanup: {e}")

## Delete Lambda Function

In [ ]:
# Delete Lambda functions (interceptor + tools)
print("\nDeleting Lambda functions...")

# Delete interceptor Lambda
try:
    lambda_client.delete_function(FunctionName=REQUEST_LAMBDA_FUNCTION_NAME)
    print(f"  ✓ Interceptor Lambda deleted: {REQUEST_LAMBDA_FUNCTION_NAME}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceNotFoundException':
        print(f"  ⚠ Interceptor Lambda not found: {REQUEST_LAMBDA_FUNCTION_NAME}")
    else:
        print(f"  ⚠ Error: {e}")

# Delete tool Lambdas
if 'DEPLOYED_TOOL_FUNCTIONS' in globals() and DEPLOYED_TOOL_FUNCTIONS:
    print(f"\n  Deleting {len(DEPLOYED_TOOL_FUNCTIONS)} tool Lambdas...")
    for function_name in DEPLOYED_TOOL_FUNCTIONS:
        try:
            lambda_client.delete_function(FunctionName=function_name)
            print(f"    ✓ Deleted: {function_name}")
        except ClientError as e:
            if e.response['Error']['Code'] == 'ResourceNotFoundException':
                print(f"    ⚠ Not found: {function_name}")
            else:
                print(f"    ⚠ Error deleting {function_name}: {e}")

## Delete IAM Role

In [ ]:
# Delete IAM roles (Lambda interceptor, tools, Gateway)
print("\nDeleting IAM roles...")

# Delete Lambda interceptor role
try:
    print("  Deleting Lambda interceptor role...")
    
    # Detach policies
    iam_client.detach_role_policy(
        RoleName=LAMBDA_ROLE_NAME,
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
    )
    
    # Delete inline policies
    iam_client.delete_role_policy(
        RoleName=LAMBDA_ROLE_NAME,
        PolicyName='DynamoDBAccess'
    )
    
    # Delete role
    iam_client.delete_role(RoleName=LAMBDA_ROLE_NAME)
    print(f"    ✓ Lambda interceptor role deleted: {LAMBDA_ROLE_NAME}")
    
except ClientError as e:
    if e.response['Error']['Code'] == 'NoSuchEntity':
        print(f"    ⚠ Role not found: {LAMBDA_ROLE_NAME}")
    else:
        print(f"    ⚠ Error: {e}")

# Delete tool Lambda role
if 'TOOL_ROLE_NAME' in globals():
    try:
        print("  Deleting tool Lambda role...")
        
        # Detach policies
        iam_client.detach_role_policy(
            RoleName=TOOL_ROLE_NAME,
            PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
        )
        
        # Delete role
        iam_client.delete_role(RoleName=TOOL_ROLE_NAME)
        print(f"    ✓ Tool Lambda role deleted: {TOOL_ROLE_NAME}")
        
    except ClientError as e:
        if e.response['Error']['Code'] == 'NoSuchEntity':
            print(f"    ⚠ Role not found: {TOOL_ROLE_NAME}")
        else:
            print(f"    ⚠ Error: {e}")

# Delete Gateway role
if 'GATEWAY_ROLE_NAME' in globals():
    try:
        print("  Deleting Gateway role...")
        
        # Detach admin policy
        iam_client.detach_role_policy(
            RoleName=GATEWAY_ROLE_NAME,
            PolicyArn='arn:aws:iam::aws:policy/AdministratorAccess'
        )
        
        # Delete role
        iam_client.delete_role(RoleName=GATEWAY_ROLE_NAME)
        print(f"    ✓ Gateway role deleted: {GATEWAY_ROLE_NAME}")
        
    except ClientError as e:
        if e.response['Error']['Code'] == 'NoSuchEntity':
            print(f"    ⚠ Role not found: {GATEWAY_ROLE_NAME}")
        else:
            print(f"    ⚠ Error: {e}")

---

# Summary

This notebook completed the full lifecycle:

1. ✅ **Setup** - Lambda, IAM Role, and Gateway
2. ✅ **Test** - Verified through AgentCore Gateway
3. ✅ **Cleanup** - Deleted all resources

## What We Demonstrated

- **Lambda REQUEST interceptor** that modifies Gateway requests
- **Transforming Schema and adding additional parameters** through the request chain
- **Complete resource lifecycle** management

## Next Steps

- Adding more schema translation use-cases
- Integrate with real AgentCore Runtime agents
- Monitor CloudWatch logs for debugging